# 最終課題

In [ ]:
# 必要なライブラリをインポート
import requests  # Webページにアクセスするためのライブラリ
from bs4 import BeautifulSoup  # HTMLを解析するためのライブラリ
from urllib.parse import urljoin, urlparse  # URLを操作するためのライブラリ
import time  # 待機時間を設定するためのライブラリ

def get_domain(url):
    """URLからドメイン名を取得"""
    # URLを解析してスキーム（http/https）とドメイン名を取得
    parsed = urlparse(url)
    # "https://www.musashino-u.ac.jp" のような形式で返す
    return f"{parsed.scheme}://{parsed.netloc}"

def is_same_domain(url, base_domain):
    """同一ドメインかチェック"""
    # 指定されたURLが基準ドメインと同じかどうかを判定
    return get_domain(url) == base_domain

def extract_sitemap(start_url, sleep_time=1):
    """
    Webサイトのサイトマップを抽出
    
    Parameters:
    start_url: 開始URL（武蔵野大学トップページ）
    sleep_time: アクセス間隔（秒）
    
    Returns:
    dict: {URL: タイトル} の辞書
    """
    # データを格納する変数を初期化
    sitemap = {}  # URLをキー、タイトルを値とする辞書型変数
    visited = set()  # 訪問済みのURLを記録するセット（重複防止）
    to_visit = [start_url]  # これから訪問するURLのリスト
    base_domain = get_domain(start_url)  # 基準となるドメイン名を取得
    
    print(f"サイトマップ抽出を開始します: {start_url}")
    print(f"ベースドメイン: {base_domain}\n")
    
    # 訪問するURLがある限りループを続ける
    while to_visit:
        # リストの先頭からURLを取り出す
        current_url = to_visit.pop(0)
        
        # 既に訪問済み、または同一ドメインでない場合はスキップ
        if current_url in visited or not is_same_domain(current_url, base_domain):
            continue
        
        try:
            # 現在アクセスしているURLを表示
            print(f"アクセス中: {current_url}")
            
            # Webページにアクセスして内容を取得（タイムアウト10秒）
            response = requests.get(current_url, timeout=10)
            # 文字エンコーディングを自動判定して設定
            response.encoding = response.apparent_encoding
            
            # ステータスコードが200（正常）でない場合はスキップ
            if response.status_code != 200:
                print(f"  ステータスコード {response.status_code} - スキップ")
                visited.add(current_url)  # 訪問済みに追加
                continue
            
            # BeautifulSoupでHTMLを解析
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # <title>タグを検索してタイトルを取得
            title_tag = soup.find('title')
            # タイトルタグがあれば中身のテキストを、なければ"タイトルなし"を設定
            title = title_tag.get_text(strip=True) if title_tag else "タイトルなし"
            
            # 辞書型変数に格納（key: URL, value: タイトル）
            sitemap[current_url] = title
            # 訪問済みセットに追加
            visited.add(current_url)
            
            # 進捗状況を表示
            print(f"  タイトル: {title}")
            print(f"  進捗: {len(visited)} ページ訪問済み\n")
            
            # ページ内の全ての<a>タグ（リンク）を取得
            links = soup.find_all('a', href=True)
            
            # 各リンクを処理
            for link in links:
                # href属性（リンク先URL）を取得
                href = link['href']
                # 相対URLを絶対URLに変換
                absolute_url = urljoin(current_url, href)
                
                # フラグメント（#以降のアンカー部分）を除去
                absolute_url = absolute_url.split('#')[0]
                
                # 同一ドメイン内で、未訪問かつ訪問予定リストにもない場合
                if (is_same_domain(absolute_url, base_domain) and 
                    absolute_url not in visited and 
                    absolute_url not in to_visit):
                    # 訪問予定リストに追加
                    to_visit.append(absolute_url)
            
            # サーバー負荷軽減のため指定秒数待機（デフォルト1秒）
            time.sleep(sleep_time)
            
        # エラーが発生した場合の処理
        except Exception as e:
            # エラー内容を表示
            print(f"  エラー: {e}\n")
            # 訪問済みに追加して次へ進む
            visited.add(current_url)
            continue
    
    # 抽出完了メッセージを表示
    print(f"抽出完了: 合計 {len(sitemap)} ページ\n")
    # 辞書型変数を返す
    return sitemap


# メイン処理（このファイルが直接実行された場合のみ実行）
if __name__ == "__main__":
    # 武蔵野大学のトップページURL
    musashino_url = "https://www.musashino-u.ac.jp/"
    
    # サイトマップを抽出（アクセス間隔: 1秒）
    sitemap_dict = extract_sitemap(musashino_url, sleep_time=1)
    
    # 結果を辞書型変数として表示
    print("=" * 80)
    print("抽出されたサイトマップ")
    print("=" * 80)
    print(sitemap_dict)  # 辞書型変数をprint()で表示
    print("\n")
    
    # 見やすく整形して表示（オプション）
    print("=" * 80)
    print("整形表示")
    print("=" * 80)
    for url, title in sitemap_dict.items():
        print(f"URL: {url}")
        print(f"Title: {title}")
        print("-" * 80)

サイトマップ抽出を開始します: https://www.musashino-u.ac.jp/
ベースドメイン: https://www.musashino-u.ac.jp

アクセス中: https://www.musashino-u.ac.jp/
  タイトル: 武蔵野大学
  進捗: 1 ページ訪問済み

アクセス中: https://www.musashino-u.ac.jp/access.html
  タイトル: 交通アクセス | 武蔵野大学
  進捗: 2 ページ訪問済み

アクセス中: https://www.musashino-u.ac.jp/admission/request.html
  タイトル: 資料請求 | 入試情報 | 武蔵野大学
  進捗: 3 ページ訪問済み

アクセス中: https://www.musashino-u.ac.jp/contact.html
  タイトル: お問い合わせ | 武蔵野大学
  進捗: 4 ページ訪問済み

アクセス中: https://www.musashino-u.ac.jp/prospective-students.html
  タイトル: 武蔵野大学で学びたい方 | 武蔵野大学
  進捗: 5 ページ訪問済み

アクセス中: https://www.musashino-u.ac.jp/students.html
  タイトル: 在学生の方 | 武蔵野大学
  進捗: 6 ページ訪問済み

アクセス中: https://www.musashino-u.ac.jp/alumni.html
  タイトル: 卒業生の方 | 武蔵野大学
  進捗: 7 ページ訪問済み

アクセス中: https://www.musashino-u.ac.jp/parents.html
  タイトル: 保護者の方 | 武蔵野大学
  進捗: 8 ページ訪問済み

アクセス中: https://www.musashino-u.ac.jp/business.html
  タイトル: 企業・研究者の方 | 武蔵野大学
  進捗: 9 ページ訪問済み

アクセス中: https://www.musashino-u.ac.jp/guide/
  タイトル: 大学案内 | 武蔵野大学
  進捗: 10 ページ訪問済み

アクセス中: ht